In [2]:

import sqlite3
import json
from typing import Literal, Generator, List, Optional
from sqlite3 import Error, PrepareProtocol, register_adapter
from pydantic import BaseModel

class Message(BaseModel):
    id: int = 0
    role: Literal["system", "user", "assistant"]
    content: str
    summary_id: int = 0

class Summary(BaseModel):
    id: int
    title: str
    summary: str
    entities: List[str]
    
class SummaryAndMessages(BaseModel):
    summary: Summary
    messages: List[Message]


In [22]:
conn = sqlite3.connect(r'Z:\electron_groq\database.db')
# conn.row_factory = sqlite3.Row
cursor = conn.cursor()

In [10]:
%load_ext sql
%sql sqlite:///Z:\electron_groq\database.db

Connecting to 'sqlite:///Z:\\electron_groq\\database.db'

In [7]:
%%sql 
SELECT * FROM sqlite_master;

Running query in 'sqlite:///Z:\\electron_groq\\database.db'

type,name,tbl_name,rootpage,sql
table,summary,summary,3,"CREATE TABLE summary ( id integer PRIMARY KEY, title text NOT NULL, summary text NOT NULL, entities text NOT NULL )"
table,master,master,5,"CREATE TABLE master (id, role, content, summary_id)"
table,temp_chat,temp_chat,2,"CREATE TABLE temp_chat ( id integer PRIMARY KEY, role text NOT NULL, content text NOT NULL )"


In [11]:
%%sql 
SELECT * FROM temp_chat;

Running query in 'sqlite:///Z:\\electron_groq\\database.db'

id,role,content
1,system,You are a super helpful assistant.
2,user,Use the online_llm function to search for the current weather conditions in the user's location.
3,assistant,"The current weather conditions in London, UK are partly cloudy with a temperature of 16°C, wind speed of 7mph, and humidity of 82%.For more detailed weather information, you can visit the Met Office website, which provides a wide range of forecasts for the UK, including regions and specific locations. The website offers hourly forecasts for the first two days and then in 3-hour time steps for the following days. It includes weather, temperature, wind speed and direction, UV and feels like temperature, and more. You can also choose the units used on your forecast display, such as Celsius or Fahrenheit for temperature.For beach forecasts, the Met Office provides an indication of whether the wind will be on-shore or off-shore, offshore sea state and tide times for the nearest available port. However, please note that the beach information is still under development and is intended to give an indication only, it should not be used for safety-critical decisions.For current weather conditions from across the UK, Netweather provides a live weather service with current weather conditions including wind speed, humidity and atmospheric pressure as well as hourly weather forecast for the current day and for the next day.For a responsive weather widget for your website, Weatherwidget.org offers a free weather widget that displays current weather conditions including wind speed, humidity and atmospheric pressure as well as hourly weather forecast for the current day and for the next day. The widget is customizable, available in 66 languages, and supports both English units and metric units."
4,user,"From the search results, identify the current time of the day (e.g., morning, afternoon, evening, or night)."
5,assistant,"Based on the search results, the current time of the day is not explicitly stated. However, weather information is typically updated frequently and often includes the time the data was last updated. From the provided information, it appears that the search results are based on current data, which would suggest that the current time is midday or afternoon. The sun is at its highest point in the sky around 12:00pm, and depending on the user's location, the time may be slightly earlier or later. It is safe to assume that the search results were generated during the daytime."


In [ ]:
cursor.execute("""CREATE TABLE IF NOT EXISTS master_s (id, role, content);""")

cursor.execute("""
    INSERT INTO master_s (id, role, content)
    SELECT id, role, content FROM master
    UNION ALL
    SELECT id, role, content FROM temp_chat;
""")

cursor.execute("DROP TABLE master")

cursor.execute("CREATE TABLE IF NOT EXISTS master (id, role, content);")

cursor.execute("""
    INSERT INTO master (id, role, content)
    SELECT id, role, content FROM master_s
""")

cursor.execute("DROP TABLE master_s")

conn.commit()

In [36]:
cursor.execute("SELECT * FROM temp_chat")
rows = cursor.fetchall()
messages = [{"role": row[1], "content": row[2]} for row in rows]  # use pydantic's [message.to_dict() for row in rows]
messages

[{'role': 'system', 'content': 'You are a super helpful assistant.'}]

In [ ]:
row = {"role": rows[0][1], "content": rows[0][2]}
Message(**row)

In [45]:
from groq import Groq
import os, json

conn.row_factory = sqlite3.Row
cursor = conn.cursor()

client = Groq(api_key=os.getenv('GROQ_API_KEY')) 

cursor.execute("SELECT * FROM temp_chat")
rows = cursor.fetchall()
messages = [Message(role=row['role'], content=row['content']) for row in rows] 

messages.append(Message(role="system", content=f"You are a summary database that outputs summaries in JSON.\n\
    The JSON object must use the schema: {json.dumps(Summary.model_json_schema(), indent=2)}"))
    
messages.append(Message(role="user", content="Summarise the above conversation and list up all named entities."))

main_request = client.chat.completions.create(model="mixtral-8x7b-32768",
                                              messages=messages,
                                              response_format={"type": "json_object"})
         
res = main_request.choices[0].message.content

summary_dict = Summary.model_validate_json(res).model_dump()

'{\n  "summary": "A request was made to summarize text and list named entities in JSON format, following a specific schema. The schema requires a summary field and an entities field, with entities being an array of strings. The AI responds that it is a super helpful assistant, ready to summarize and list entities in the requested format.",\n  "entities": [\n    "JSON",\n    "schema"\n  ]\n}'

In [64]:
summary_dict['summary']

'A request was made to summarize text and list named entities in JSON format, following a specific schema. The schema requires a summary field and an entities field, with entities being an array of strings. The AI responds that it is a super helpful assistant, ready to summarize and list entities in the requested format.'

In [49]:
cursor.execute("""SELECT * FROM summary;""")
res = cursor.fetchall()
res

[(1,
  'The conversation covers a class that manages a SQLite database connection, including methods for creating a table, inserting data, dropping a table, and loading data from the database. The class maintains a cursor object that is shared within the class, with a discussion of the potential benefits and drawbacks of sharing the cursor.',
  'LocalDB, sqlite3, Message, create_table, insert_data, load_temp, drop_table, load_master, cursor, fetchall, fetchmany, fetchone, commit, row_factory, rollback, Connection, Error, Thread, multiprocessing, concurrent.futures, concurrent.executors, concurrent.futures.ThreadPoolExecutor, concurrent.futures.ProcessPoolExecutor')]

In [26]:
cursor.execute("""SELECT MAX(id) FROM summary;""")
res = cursor.fetchone()
res[0]

1

In [69]:
cursor.execute("DROP TABLE temp_chat")
cursor.execute("DROP TABLE master")
# cursor.execute("DROP TABLE master_s")
cursor.execute("""CREATE TABLE IF NOT EXISTS master (
    id integer, role text, content text, summary_id integer);""")

cursor.execute("DROP TABLE summary")
cursor.execute("""CREATE TABLE IF NOT EXISTS summary (
                            id integer PRIMARY KEY,
                            title text NOT NULL,
                            summary text NOT NULL, 
                            entities text NOT NULL
                            );""")
conn.commit()

In [ ]:
conn.close()

### using [adaptors](https://docs.python.org/3/library/sqlite3.html#how-to-adapt-custom-python-types-to-sqlite-values) to conform objects.

In [31]:
class Point:
    def __init__(self, x, y):
        self.x, self.y = x, y

    def __conform__(self, protocol):
        if protocol is sqlite3.PrepareProtocol:
            return f"{self.x};{self.y}"

con = sqlite3.connect(":memory:")
cur = con.cursor()

cur.execute("SELECT ?", (Point(4.0, -3.2),))
print(cur.fetchone()[0])

4.0;-3.2


In [ ]:
def adapt(summary):
    return ' '.join(summary.entities)
    
register_adapter(Summary, adapt)

In [ ]:
def adapt_list_to_json(lst):
    return json.dumps(lst).encode('utf-8')

def convert_json_to_list(text):
    return json.loads(text.decode('utf-8'))

sqlite3.register_adapter(list, adapt_list_to_json)
sqlite3.register_converter("JSON", convert_json_to_list)

# Connect to your SQLite database, enabling the converter
conn = sqlite3.connect('database.db', detect_types=sqlite3.PARSE_DECLTYPES)
cursor = conn.cursor()

cursor.execute("""
            INSERT INTO summary (summary, entities) 
            VALUES (?, ?);""", (summary.summary, summary.entities))

# Don't forget to commit and close
conn.commit()
conn.close()

In [60]:
summary.entities

['JSON', 'schema']

In [15]:
cursor.execute("DROP TABLE temp_chat")

cursor.execute("""CREATE TABLE IF NOT EXISTS temp_chat (
                                id integer PRIMARY KEY,
                                role text NOT NULL,
                                content text NOT NULL                                
                            );""")
conn.commit()

In [21]:
# add a column
summary_id = 1
cursor.execute(f"""
               ALTER TABLE temp_chat
               ADD COLUMN summary_id integer DEFAULT {summary_id};""")
conn.commit()
# cursor.fetchall()

In [ ]:
# set a value
summary_id = 1 
cursor.execute("""
UPDATE temp_chat
SET summary_id = ?;
""", (summary_id,))

In [62]:
cursor.execute("DROP TABLE master")
cursor.execute("""CREATE TABLE IF NOT EXISTS master (
    id integer PRIMARY KEY, 
    role text, 
    content text, 
    summary_id integer);""")

cursor.execute("DROP TABLE summary")
cursor.execute("""CREATE TABLE IF NOT EXISTS summary (
                            id integer PRIMARY KEY,
                            summary text NOT NULL, 
                            entities text NOT NULL
                            );""")
conn.commit()

In [63]:
cursor.execute("""CREATE TABLE IF NOT EXISTS temp_chat (
                                id integer PRIMARY KEY,
                                role text NOT NULL,
                                content text NOT NULL                                
                            );""")
conn.commit()

In [15]:
%%sql
SELECT COUNT(*) FROM temp_chat;

Running query in 'sqlite:///Z:\\electron_groq\\database\\database.db'

COUNT(*)
11


In [109]:
%%sql
SELECT * FROM summary;
-- WHERE id == 1;

Running query in 'sqlite:///Z:\\electron_groq\\database\\database.db'

id,title,summary,entities
1,Limits of Knowledge,"Inquiry about the limitations of the AI's knowledge. Response includes explanation of the AI's knowledge limits, such as lack of personal experiences and inability to provide professional advice.","AI, medicine, law, mental health, internet, data"
2,Romance Story Summary,"Lily, a florist, meets Alex, an artist, and they fall in love. They marry, honeymoon, and start a family. Facing challenges, but supporting each other, they live happily ever after.","Lily, Alex, Harmony, flower shop, Alex's art gallery, wedding, honeymoon, children (son, daughter)"
3,Enabling remote module for preload in Electron,"We discussed enabling the remote module for preload to load in Electron and a code snippet for creating a popup window. Named entities: Electron, preload, remote module, renderer process, web page, webPreferences, popup window, nodeIntegration, path, contextIsolation.","Electron, preload, remote module, renderer process, web page, webPreferences, popup window, nodeIntegration, path, contextIsolation"


In [47]:
cursor.execute("""SELECT * FROM summary 
               WHERE id == ?;""", (1,))
conn.commit()

In [3]:
class YesOrNo(BaseModel):
    answer: bool
    reason: str
    userInputRequired: Optional[bool]

In [9]:
YesOrNo.model_json_schema()

{'properties': {'answer': {'title': 'Answer', 'type': 'boolean'},
  'reason': {'title': 'Reason', 'type': 'string'},
  'userInputRequired': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}],
   'title': 'Userinputrequired'}},
 'required': ['answer', 'reason', 'userInputRequired'],
 'title': 'YesOrNo',
 'type': 'object'}